## 4. Lemmatisieren des Korpus
Vorbereitung für das Topic-Modelling mit LDA.

Basiert auf dem gensim Tutorial: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

### Pakete Installieren
Als erstes müssen wir die nachfolgenden Pakete installieren. `HanTa` (HannoverTagger) nutzen wir zur Lemmatisierung deutscher Texte. `gensim` bietet diverese Algorithmen zum Topic Modelling; `pyLDAvis` erlaubt die Visualisierung der Ergebnisse des LDA-Algorithmus im Jupyter-Notebook.

In [ ]:
!pip install HanTa

In [ ]:
!pip install gensim

In [ ]:
!pip install pyLDAvis

### Einlesen der Dateien & Vorbereitung

In [188]:
# Reading files into docs
files = ["data/fdp_b.txt", "data/gruen_b.txt", "data/spd_b.txt", "data/koav_b.txt"]

docs = []
for file in files:
    with open(file, 'r') as f:
        corpus = f.read()
        docs.append(corpus)

In den nächsten beiden Schritten werden alle Wörtern zur Kleinschreibung überführt `w.lower()`. Danach werden die Dokumente tokenisiert und anschließend mit Stoppwortlisten abgeglichen (genau genommen sind ja die Entfernung von Satzzeichen auch eine Art Stoppwortliste). Mit `stop_words.extend(['freie', 'demokraten', 'spd', 'innen'])` fügen wir der Standard Stoppwortliste noch künstlich Wörter hinzu, die extrem oft vorkommen, da sie für die Topics keine Bedeutung haben.

In [189]:
docs = [w.lower() for w in docs]

In [190]:
# Tokenize the documents.
# Source: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
from nltk.tokenize import RegexpTokenizer
import requests 


r = requests.get('https://github.com/stopwords-iso/stopwords-de/raw/master/stopwords-de.json')
stop_words = r.json()
stop_words.extend(['freie', 'demokraten', 'spd', 'innen'])

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
    

# Removing Stopwords
docs = [[w for w in doc if not w.lower() in stop_words] for doc in docs] # Removing Stopwords

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

### Lemmatisierung
Nun folgt die Lemmatisierung mit dem `HanTa`-Paket. Der Code wurde aus Stackoverflow übernommen:  https://stackoverflow.com/questions/57857240/ho-to-do-lemmatization-on-german-text

**Achtung** Die Ausführung der nachfolgenen Zeile nimmt einige Zeit in Anspruch!

In [192]:
# Taken from: https://stackoverflow.com/questions/57857240/ho-to-do-lemmatization-on-german-text
from HanTa import HanoverTagger as ht

tagger = ht.HanoverTagger('morphmodel_ger.pgz')

docs_lemma = []
docs = [[lemma for (word,lemma,pos) in tagger.tag_sent(doc)] for doc in docs]

### Griff in die Trickkiste: Von 4 zu > 1100 Dokumenten

Da wir nur eine sehr geringe Zahl an Dokumenten (4) haben, werden die Ergebnisse nicht sehr aussagekräftig sein. Immerhin versuchen vermutlich alle Parteien ein möglichst breites Spektrum an Themen abzudecken (wie auch der KoaV). Deshalb teilen wir im folgenden Schritt die Texte künstlich auf. Für eine realistische Analyse muss hier natürich hinterfragt werden wie sinnvoll diese Umsetzung ist, aber man könnte diese Frage auch früher ansetzen: Warum sehen wir uns nicht einfach die Überschriften der Kapitel an um Topics zu erhalten. Ein realistischer Einsatz des Topic Modellings wäre z. B. das Sortieren von Wikipedia-Artikeln oder Zeitungsartikeln, bei denen wir eben nicht unbedingt wissen was darin enthalten ist. Um aber für die Vorlesung bei einem Beispiel zu bleiben wenden wir den nachfolgenden Schritt an. 

Da ich die Wort-Zahl pro Chunk beim Testen variieren wollte habe ich `docs` in eine `docs_backup` kopiert, damit ich zum Testen nicht immer auf die langwierige Lemmatisierung warten muss.

In [193]:
docs_backup = docs

In [211]:
# Since we have only few documents and first test runs yield bad results we're trying to create more documents by splitting each into evenly sized chunks
# Source: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
dd = []
for doc in docs_backup:
    doc_chunks = chunks(doc, 75) # Splitting into 100 token long chunks
    for doc_chunk in doc_chunks:
        dd.append(doc_chunk)
        
docs = dd

### Gensim: Bigrams
Nach dem splitten der Dokumente in Chunks fahren wir mit dem `gensim`-Tutorial fort.
Quelle: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

**Bigrams**: Aufteilung des Textes in Bi- und Trigramme. Damit sind die Wortfolgen von zwei, bzw. drei Wörtern gemeint.

In [212]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [220]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents
dictionary.filter_extremes(no_below=20)

In [221]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [222]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 735
Number of documents: 1193


### LDA Training
Nun sind alle Vorbereitung abgeschlossen. Wir haben ca. 1200 Dokumente mit 7089 eindeutigen Tokens. Diese werden nun in den LDA-Algorithmus gegeben und daraus ein Modell trainiert.

**Achtung** auch dieser Schritt kann, abhängig von der Rechnerleistung, etwas länger in Anspruch nehmen.

In [223]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 40
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

Die folgenden Zeilen geben die Topics in Textform aus.

In [224]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -2.1025.
[([(0.050007112, 'europäisch'),
   (0.045915015, 'Eu'),
   (0.022638977, 'international'),
   (0.017420039, 'gemeinsam'),
   (0.016503593, 'Europa'),
   (0.016323822, 'stärken'),
   (0.014724072, 'setzen'),
   (0.012801472, 'Deutschland'),
   (0.011953, 'Menschenrecht'),
   (0.011932265, 'unterstützen'),
   (0.011245969, 'deutsch'),
   (0.0104221245, 'Staat'),
   (0.009827345, 'Demokratie'),
   (0.009240756, 'Zusammenarbeit'),
   (0.008814832, 'global'),
   (0.008326743, 'stark'),
   (0.008131975, 'Sicherheit'),
   (0.007849944, 'politisch'),
   (0.007648137, 'Partner'),
   (0.0076194345, 'demokratisch')],
  -1.7962923049073334),
 ([(0.036392592, 'Land'),
   (0.022430167, 'digital'),
   (0.021775171, 'Bund'),
   (0.018608214, 'öffentlich'),
   (0.017336227, 'stärken'),
   (0.015363795, 'brauchen'),
   (0.014967956, 'Kommune'),
   (0.013710678, 'gemeinsam'),
   (0.012243469, 'Bund_Land'),
   (0.012062959, 'stellen'),
   (0.011027842, 'schaffen'),
   (0.

### Visualisierung
Interessanter ist aber die Interaktive Visualisierung mit `pyLDAvis`. Auf der linken seite sehen wir die Abstände zwischen den jeweiligen Topics, mit der Maus ausgewählt sehen wir rechts die meist-relevanten Wörter für das jeweilige Topic. Mit dem Slider können wir noch die *relevance metric* anpassen.

Quelle: https://nbviewer.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb#topic=8&lambda=1&term=

In [225]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
gensimvis.prepare(model, corpus, dictionary)

/home/jupyter-michael/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.137702 -0.058360       1        1  17.687297
9      0.033206 -0.117118       2        1  13.306142
5     -0.105666  0.133027       3        1  11.727177
7     -0.041190 -0.005325       4        1  10.274387
0      0.025019  0.014347       5        1   9.955836
6      0.182215  0.108044       6        1   8.887146
8     -0.174090  0.089926       7        1   8.841153
4      0.004924 -0.028827       8        1   7.379429
3     -0.122140 -0.162818       9        1   7.342190
1      0.060020  0.027105      10        1   4.599244, topic_info=                 Term        Freq       Total Category  logprob  loglift
372              Kind  260.000000  260.000000  Default  30.0000  30.0000
260                Eu  487.000000  487.000000  Default  29.0000  29.0000
194        europäisch  580.000000  580.000000  Default  28.0000  28.0000
12            digital  385.000000  385.000000  Default  27.0000  27.0000
4                Land  413.000000  413.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
329       ermöglichen   18.798927  276.901666  Topic10  -4.7502   0.3894
0              Chance   15.364406  137.083893  Topic10  -4.9519   0.8907
257           Bereich   14.190014  126.895930  Topic10  -5.0315   0.8884
220      unterstützen   15.917595  398.399589  Topic10  -4.9166  -0.1408
305  weiterentwickeln   13.960330  133.775013  Topic10  -5.0478   0.8193

[549 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
285       1  0.769617     Abkommen
285       2  0.185770     Abkommen
285       9  0.026539     Abkommen
379       3  0.238766  Absicherung
379       7  0.682190  Absicherung
...     ...       ...          ...
239       1  0.036394   überwinden
239       5  0.072787   überwinden
239       6  0.582298   überwinden
239       9  0.109181   überwinden
239      10  0.181968   überwinden

[2040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 6, 8, 1, 7, 9, 5, 4, 2])